<a href="https://colab.research.google.com/github/simonbustamante/mit-algoritmos-de-regresion/blob/master/Caso_de_Estudio_2_4_Predicci%C3%B3n_de_salarios_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 2.4 - Predicción de salarios II

---



Configuración del *notebook*:

Sincronice su cuenta de Google. Para ello, siga el link que aparece en la salida de la siguiente celda una vez ejecutada. Copie el código que le aparece en pantalla e introdúzcalo en la salida de la celda. Una vez vea el mensaje: `Google Drive sincronizado con éxito!` puede continuar ejecutando el resto de celdas.

In [1]:
from google.colab import auth
auth.authenticate_user()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
data_drop = drive.CreateFile({'id':'1uQj3lEsilNJzwxWNn-PBM4exOUTBDFy7'})
data_drop.GetContentFile('wage2015.Rdata')

print('Google Drive sincronizado con éxito!')

Google Drive sincronizado con éxito!


In [2]:
!pip install rpy2==3.5.1
%load_ext rpy2.ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp310-cp310-linux_x86_64.whl size=314932 sha256=ca196c414da5a8e6847988e725eda8f800b00a235490c11273314d4c2941305f
  Stored in directory: /root/.cache/pip/wheels/73/a6/ff/4e75dd1ce1cfa2b9a670cbccf6a1e41c553199e9b25f05d953
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.4.2
    Uninstalling rpy2-3.4.2:
      Successfully uninstalled rpy2-3.4.2


Instalar e importar librerías: (ignorar resultados a no ser que no se muestre la frase: `Librerías instaladas con éxito!`)

In [3]:
%%R --noreturn
# Instalar librerías
install.packages("hdm")
install.packages("randomForest")
install.packages("glmnet")
install.packages("nnet")
install.packages("rpart")
install.packages("gbm")
install.packages("rpart.plot")

cat('\n¡Librerías instaladas con éxito!')

(as ‘lib’ is unspecified)

















































	‘/tmp/RtmpbH2GZ2/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmpbH2GZ2/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmpbH2GZ2/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmpbH2GZ2/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmpbH2GZ2/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmpbH2GZ2/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmpbH2GZ2/downloaded_packages’




¡Librerías instaladas con éxito!

In [4]:
%%R
# Cargar librerías
library(hdm)
library(randomForest)
library(glmnet)
library(nnet)
options(warn=-1)
library(rpart)
library(gbm)
library(rpart.plot)

cat('\n¡Librerías importadas con éxito!')


¡Librerías importadas con éxito!

## Datos


In [5]:
%%R
# Cargar la base de datos
load(file="wage2015.Rdata")

# Ver las variables de la base de datos
class(data)
str(data)

# Mostrar dimensiones de la base de datos
dims  <- dim(data)
cat('\nDimensiones de la base de datos:',toString(dims),'\n',fill = TRUE)

'data.frame':	12697 obs. of  37 variables:
 $ wage    : num  9.62 48.08 11.06 11.54 13.94 ...
 $ lwage   : num  2.26 3.87 2.4 2.45 2.63 ...
 $ sex     : num  0 1 1 1 0 0 0 0 0 1 ...
 $ white   : num  1 1 1 1 1 1 0 1 1 1 ...
 $ black   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hisp    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ shs     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hsg     : num  0 0 1 0 0 0 0 0 1 1 ...
 $ scl     : num  0 0 0 1 0 0 0 0 0 0 ...
 $ clg     : num  1 1 0 0 0 1 1 1 0 0 ...
 $ mw      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ union   : num  0 0 0 0 0 1 0 0 0 0 ...
 $ vet     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ cent    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ncent   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ fam1    : num  0 1 1 1 1 1 1 1 1 1 ...
 $ fam2    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ fam3    : num  1 0 0 0 0 0 0 0 0 0 ...
 $ child   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ fborn   : num  0 0 0 0 0 0 1 0 0 0 ...
 $ cit     : num  1 

Separación de los datos en un conjunto de entrenamiento y otro de prueba

In [6]:
%%R
# Generador de números aleatorios
set.seed(1)
# Conjunto de índices de entrenamiento
training <- sample(nrow(data), nrow(data)*(1/2), replace=FALSE)
# Conjunto de entrenamiento
datause <- data[training,]
# Conjunto de prueba
dataout <- data[-training,]

## Metodología

### Definición de modelos (básico y flexible)

In [7]:
%%R
# Variables de control lineales. Usar estas variables de control para métodos básasados en árboles de regresión
x <- "sex+white+black+hisp+shs+hsg+scl+clg+mw+so+we+union+vet+cent+ncent+fam1+fam2+fam3+child+\
fborn+cit+school+pens+fsize10+fsize100+health+age+exp1+occ2+ind2"

# Variables de control cuadráticas (especificación flexible). Usar estas variables de control para métodos lineales
xL <- "(sex+white+black+hisp+shs+hsg+scl+clg+mw+so+we+union+vet+cent+ncent+fam1+fam2+fam3+child+\
fborn+cit+school+pens+fsize10+fsize100+health+age+exp1+exp2+exp3+exp4+occ2+ind2)^2"

# variable de resultado: log wage (logaritmo del salario)
y  <- "lwage"

# Modelo lineal: Especificación cuadrática
formL <- as.formula(paste(y, "~", xL))
# Modelo lineal: Especificación lineal
form  <- as.formula(paste(y, "~", x))

In [8]:
%%R
# "-1" : no incluir la constante en el modelo
# x,y TRUE: devolver la matriz de covariables/vector de resultados
# Ejecute estas regresiones lineales para usar sus variables de resultado (fituse$y) y covariables (fituse$y)
# un truco para extraer las variables x e y de una fórmula

fituseL    <- lm(paste(y, "~", xL, "-1"), datause, x=TRUE, y=TRUE)
fitoutL    <- lm(paste(y, "~", xL, "-1"), dataout, x=TRUE, y=TRUE)
fituse     <- lm(paste(y, "~", x,  "-1"), datause, x=TRUE, y=TRUE)
fitout     <- lm(paste(y, "~", x,  "-1"), dataout, x=TRUE, y=TRUE)

### Entrenamiento con métodos lineales y no lineales

In [9]:
%%R
start_time <- Sys.time()

# Regresión lineal modelo simple
fit.lm      <- lm(form, datause)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento OLS: %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento OLS: 0.04 s"


In [10]:
%%R
start_time <- Sys.time()

# Regresión lineal modelo flexible
fit.lm2     <- lm(formL, datause)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento OLS (flexible): %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento OLS (flexible): 25.84 s"


In [11]:
%%R
start_time <- Sys.time()

#Lasso
fit.rlasso   <- rlasso(form, datause, post=FALSE)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Lasso: %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento Lasso: 0.56 s"


In [12]:
%%R
start_time <- Sys.time()

#Post-Lasso
fit.rlasso2  <- rlasso(form, datause, post=TRUE)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Post-Lasso: %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento Post-Lasso: 0.50 s"


Configuración de regresiones penalizadas:

* alpha=1: norma 1 (Lasso), alpha=0: norma 2 (Ridge)
* alpha = 0.5 : ambas penalizaciones (Elastic Net)

* post = FALSE: sin volver a ejecutar mínimos cuadrados en las variables seleccionadas

In [13]:
%%R
start_time <- Sys.time()

#Lasso con Validación Cruzada (VC)
fit.lasso    <- cv.glmnet(fituse$x, fituse$y, family="gaussian", alpha=1)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Lasso con VC: %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento Lasso con VC: 0.60 s"


In [14]:
%%R
start_time <- Sys.time()

#Ridge con VC
fit.ridge    <- cv.glmnet(fituse$x, fituse$y, family="gaussian", alpha=0)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Ridge con VC: %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento Ridge con VC: 0.40 s"


In [15]:
%%R
start_time <- Sys.time()

#Elastic Net con VC
fit.elnet    <- cv.glmnet(fituse$x, fituse$y, family="gaussian", alpha=.5)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Elastic Net con VC: %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento Elastic Net con VC: 0.39 s"


ATENCIÓN: la siguiente celda podría tardar unos 4 minutos en ejecutarse

In [16]:
%%R
start_time <- Sys.time()

#Lasso (flexible)
fit.rlassoL  <- rlasso(formL, datause, post=FALSE)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Lasso (flexible): %.2f min',difftime(end_time,start_time,units = 'mins'))

[1] "Tiempo de entrenamiento Lasso (flexible): 4.22 min"


ATENCIÓN: la siguiente celda podría tardar unos 2 minutos en ejecutarse

In [17]:
%%R
start_time <- Sys.time()

#Post-Lasso (flexible)
fit.rlasso2L <- rlasso(formL, datause, post=TRUE)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Post-Lasso (flexible): %.2f mins',difftime(end_time,start_time,units = 'mins'))

[1] "Tiempo de entrenamiento Post-Lasso (flexible): 1.60 mins"


ATENCIÓN: la siguiente celda podría tardar unos 14 minutos en ejecutarse

In [18]:
%%R
start_time <- Sys.time()

#Lasso con VC (flexible)
fit.lassoL   <- cv.glmnet(fituseL$x, fituseL$y, family="gaussian", alpha=1)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Lasso con VC (flexible): %.2f mins',difftime(end_time,start_time,units = 'mins'))

[1] "Tiempo de entrenamiento Lasso con VC (flexible): 8.49 mins"


ATENCIÓN: la siguiente celda podría tardar unos 2 minutos en ejecutarse

In [19]:
%%R
start_time <- Sys.time()

#Ridge con VC (flexible)
fit.ridgeL   <- cv.glmnet(fituseL$x, fituseL$y, family="gaussian", alpha=0)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Ridge con VC (flexible): %.2f mins',difftime(end_time,start_time,units = 'mins'))

[1] "Tiempo de entrenamiento Ridge con VC (flexible): 1.44 mins"


ATENCIÓN: la siguiente celda podría tardar unos 13 minutos en ejecutarse

In [20]:
%%R
start_time <- Sys.time()

#Elastic Net con VC (flexible)
fit.elnetL   <- cv.glmnet(fituseL$x, fituseL$y, family="gaussian", alpha=.5)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Elastic Net con VC (flexible): %.2f mins',difftime(end_time,start_time,units = 'mins'))

[1] "Tiempo de entrenamiento Elastic Net con VC (flexible): 7.48 mins"


ATENCIÓN: la siguiente celda podría tardar unos 4 minutos en ejecutarse

In [21]:
%%R
start_time <- Sys.time()

#Random Forest
fit.rf       <- randomForest(form, ntree=2000, nodesize=5, data=datause)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Random Forest: %.2f mins',difftime(end_time,start_time,units = 'mins'))

[1] "Tiempo de entrenamiento Random Forest: 4.39 mins"


In [22]:
%%R
start_time <- Sys.time()

#Boosting Trees
fit.boost    <- gbm(form, data=datause, distribution= "gaussian", bag.fraction = .5, interaction.depth=2, n.trees=1000, shrinkage=.01)
best.boost   <- gbm.perf(fit.boost, plot.it = FALSE)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Boosting Trees: %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento Boosting Trees: 6.15 s"


In [23]:
%%R
start_time <- Sys.time()

#Regression Trees (Árboles de Regresión, en este caso podados)
fit.trees    <- rpart(form, datause)
bestcp       <- trees$cptable[which.min(trees$cptable[,"xerror"]),"CP"]
fit.prunedtree <- prune(fit.trees,cp=bestcp)

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Árbol podado: %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento Árbol podado: 0.42 s"


In [24]:
%%R
start_time <- Sys.time()

#Red Neuronal
fit.nnet     <- nnet(form, datause, size=5,  maxit=1000, MaxNWts=100000, decay=0.01, linout = TRUE, trace=FALSE)   # ajuste simple de una red neuronal

end_time <- Sys.time()
sprintf('Tiempo de entrenamiento Red Neuronal: %.2f s',difftime(end_time,start_time,units = 'secs'))

[1] "Tiempo de entrenamiento Red Neuronal: 16.68 s"


Cálculo de las predicciones fuera de la muestra:


In [25]:
%%R

yhat.lm       <- predict(fit.lm,        newdata= dataout)
yhat.lm2      <- predict(fit.lm2,       newdata= dataout)
yhat.rlasso   <- predict(fit.rlasso,    newdata= dataout)
yhat.rlasso2  <- predict(fit.rlasso2,   newdata= dataout)
yhat.lasso    <- predict(fit.lasso,     newx   = fitout$x)
yhat.ridge    <- predict(fit.ridge,     newx   = fitout$x)
yhat.elnet    <- predict(fit.elnet,     newx   = fitout$x)
yhat.rlassoL  <- predict(fit.rlassoL,   newdata= dataout)
yhat.rlasso2L <- predict(fit.rlasso2L,  newdata= dataout)
yhat.lassoL   <- predict(fit.lassoL,    newx   = fitoutL$x)
yhat.ridgeL   <- predict(fit.ridgeL,    newx   = fitoutL$x)
yhat.elnetL   <- predict(fit.elnetL,    newx   = fitoutL$x)
yhat.rf       <- predict(fit.rf,        newdata= dataout)
yhat.boost    <- predict(fit.boost,     newdata= dataout, n.trees=best.boost)
yhat.pt       <- predict(fit.prunedtree,newdata= dataout)
yhat.nnet     <- predict(fit.nnet,      newdata= dataout)

Cálculo del EMC para cada modelo:

In [26]:
%%R
y.test       = dataout$lwage
MSE.lm       = summary(lm((y.test-yhat.lm)^2~1))$coef[1:2]
MSE.lm2      = summary(lm((y.test-yhat.lm2)^2~1))$coef[1:2]
MSE.rlasso   = summary(lm((y.test-yhat.rlasso)^2~1))$coef[1:2]
MSE.rlasso2  = summary(lm((y.test-yhat.rlasso2)^2~1))$coef[1:2]
MSE.lasso    = summary(lm((y.test-yhat.lasso)^2~1))$coef[1:2]
MSE.ridge    = summary(lm((y.test-yhat.ridge)^2~1))$coef[1:2]
MSE.elnet    = summary(lm((y.test-yhat.elnet)^2~1))$coef[1:2]
MSE.rlassoL  = summary(lm((y.test-yhat.rlassoL)^2~1))$coef[1:2]
MSE.rlasso2L = summary(lm((y.test-yhat.rlasso2L)^2~1))$coef[1:2]
MSE.lassoL   = summary(lm((y.test-yhat.lassoL)^2~1))$coef[1:2]
MSE.ridgeL   = summary(lm((y.test-yhat.ridgeL)^2~1))$coef[1:2]
MSE.elnetL   = summary(lm((y.test-yhat.elnetL)^2~1))$coef[1:2]
MSE.rf       = summary(lm((y.test-yhat.rf)^2~1))$coef[1:2]
MSE.boost    = summary(lm((y.test-yhat.boost)^2~1))$coef[1:2]
MSE.pt       = summary(lm((y.test-yhat.pt)^2~1))$coef[1:2]
MSE.nnet     = summary(lm((y.test-yhat.nnet)^2~1))$coef[1:2]

Guardar los resultados en una tabla:

In [27]:
%%R
table          <- matrix(0, 16, 3)
table[1,1:2]   <- MSE.lm
table[2,1:2]   <- MSE.lm2
table[3,1:2]   <- MSE.rlasso
table[4,1:2]   <- MSE.rlassoL
table[5,1:2]   <- MSE.rlasso2
table[6,1:2]   <- MSE.rlasso2L
table[7,1:2]   <- MSE.lasso
table[8,1:2]   <- MSE.lassoL
table[9,1:2]   <- MSE.ridge
table[10,1:2]  <- MSE.ridgeL
table[11,1:2]  <- MSE.elnet
table[12,1:2]  <- MSE.elnetL
table[13,1:2]  <- MSE.rf
table[14,1:2]  <- MSE.boost
table[15,1:2]  <- MSE.pt
table[16,1:2]  <- MSE.nnet

table[1,3]   <- 1-MSE.lm[1]/var(y.test)
table[2,3]   <- 1-MSE.lm2[1]/var(y.test)
table[3,3]   <- 1-MSE.rlasso[1]/var(y.test)
table[4,3]   <- 1-MSE.rlassoL[1]/var(y.test)
table[5,3]   <- 1-MSE.rlasso2[1]/var(y.test)
table[6,3]   <- 1-MSE.rlasso2L[1]/var(y.test)
table[7,3]   <- 1-MSE.lasso[1]/var(y.test)
table[8,3]   <- 1-MSE.lassoL[1]/var(y.test)
table[9,3]   <- 1-MSE.ridge[1]/var(y.test)
table[10,3]  <- 1-MSE.ridgeL[1]/var(y.test)
table[11,3]  <- 1-MSE.elnet[1]/var(y.test)
table[12,3]  <- 1-MSE.elnetL[1]/var(y.test)
table[13,3]  <- 1-MSE.rf[1]/var(y.test)
table[14,3]  <- 1-MSE.boost[1]/var(y.test)
table[15,3]  <- 1-MSE.pt[1]/var(y.test)
table[16,3]  <- 1-MSE.nnet[1]/var(y.test)


# Asignar nombres a columnas y filas
colnames(table)<- c("EMC", "Error Est. del EMC", "R^2")
rownames(table)<- c("Mínimos Cuadrados", "Mínimos Cuadrados (flexible)", "Lasso", "Lasso (flexible)", "Post-Lasso",  "Post-Lasso (flexible)",
                    "Lasso con VC", "Lasso con VC (flexible)", "Ridge con VC", "Ridge con VC (flexible)", "Elastic Net con VC", "Elastic Net con VC (Flexible)",
                    "Random Forest","Boosted Trees", "Árbol podado", "Red Neuronal")

### Agregación de predictores

In [28]:
%%R
# Regresión de la variable de resultado con los predictores de cada método: OLS
ens  <- lm(y.test~ yhat.lm+ yhat.rlasso+ yhat.elnet + yhat.rf+ yhat.pt +yhat.boost)
# # Regresión de la variable de resultado con los predictores de cada método: Lasso, post=FALSE
ens2 <- rlasso(y.test~ yhat.lm+ yhat.rlasso+ yhat.elnet + yhat.rf+ yhat.pt + yhat.boost, post=FALSE)

# EMC para aprendizaje agregado
MSE.ens1  <- summary(lm((y.test-ens$fitted.values)^2~1))$coef[1:2]
MSE.ens2  <- summary(lm((y.test-predict(ens2))^2~1))$coef[1:2]

In [29]:
%%R
# Tabla de resultados de aprendizaje agregado ("ensemble learning")
table2<- matrix(0, 7, 2)

table2[1,1]  <- ens$coefficients[1]
table2[2,1]  <- ens$coefficients[2]
table2[3,1]  <- ens$coefficients[3]
table2[4,1]  <- ens$coefficients[4]
table2[5,1]  <- ens$coefficients[5]
table2[6,1]  <- ens$coefficients[6]
table2[7,1]  <- ens$coefficients[7]


table2[1,2]  <- ens2$coefficients[1]
table2[2,2]  <- ens2$coefficients[2]
table2[3,2]  <- ens2$coefficients[3]
table2[4,2]  <- ens2$coefficients[4]
table2[5,2]  <- ens2$coefficients[5]
table2[6,2]  <- ens2$coefficients[6]
table2[7,2]  <- ens2$coefficients[7]

# Asignar nombres a columnas y filas
colnames(table2)<- c("Coeficiente (OLS)", "Coeficiente (Lasso)")
rownames(table2)<- c("Constante","OLS-simple","Lasso","Elastic Net con VC", "Random Forest", "Árbol podado","Boosted Trees")

## Resultados

In [30]:
%%R
# Mostrar resultados
print(table, digits=3)
print(table2, digits=3)

                                EMC Error Est. del EMC     R^2
Mínimos Cuadrados             0.293             0.0243  0.3027
Mínimos Cuadrados (flexible)  1.159             0.2382 -1.7547
Lasso                         0.301             0.0241  0.2844
Lasso (flexible)              0.299             0.0240  0.2900
Post-Lasso                    0.304             0.0242  0.2780
Post-Lasso (flexible)         0.305             0.0240  0.2753
Lasso con VC                  0.306             0.0241  0.2717
Lasso con VC (flexible)       0.308             0.0240  0.2680
Ridge con VC                  0.303             0.0241  0.2793
Ridge con VC (flexible)       0.405             0.0241  0.0381
Elastic Net con VC            0.307             0.0241  0.2693
Elastic Net con VC (Flexible) 0.309             0.0240  0.2661
Random Forest                 0.292             0.0242  0.3065
Boosted Trees                 0.296             0.0242  0.2967
Árbol podado                  0.340             0.0248 